In [ ]:
import cv2
import os

video_path = "/content/Himanshu_New_Dataset.mp4"  # your dataset video
output_folder = "dataset/images"

os.makedirs(output_folder, exist_ok=True)
cap = cv2.VideoCapture(video_path)

frame_id = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_id % 5 == 0:  # take 1 frame every 5 frames
        cv2.imwrite(f"{output_folder}/frame_{frame_id}.jpg", frame)
    frame_id += 1

cap.release()


In [ ]:
import shutil
# Zip the dataset folder
shutil.make_archive("/content/dataset", 'zip', "/content/dataset")

# Download the dataset zip (only in Google Colab)
try:
    from google.colab import files
    files.download("/content/dataset.zip")
except:
    print("Not running in Colab. Zip file saved at /content/dataset.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Another Approach

In [ ]:
pip install opencv-python mediapipe ultralytics scikit-learn


INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


Step 1: Labelled Images in Alert and Drowsy in Yolo_dataset folder

In [ ]:
import cv2
import mediapipe as mp
import os

# ===============================
# STEP 1: Setup MediaPipe & EAR function
# ===============================
mp_face_mesh = mp.solutions.face_mesh

# Left & right eye landmark indices (MediaPipe standard)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

def euclidean_dist(pt1, pt2):
    return ((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2) ** 0.5

def compute_EAR(landmarks, eye_points, w, h):
    # Get coordinates
    p1 = (landmarks[eye_points[0]].x * w, landmarks[eye_points[0]].y * h)
    p2 = (landmarks[eye_points[1]].x * w, landmarks[eye_points[1]].y * h)
    p3 = (landmarks[eye_points[2]].x * w, landmarks[eye_points[2]].y * h)
    p4 = (landmarks[eye_points[3]].x * w, landmarks[eye_points[3]].y * h)
    p5 = (landmarks[eye_points[4]].x * w, landmarks[eye_points[4]].y * h)
    p6 = (landmarks[eye_points[5]].x * w, landmarks[eye_points[5]].y * h)

    # EAR formula
    ear = (euclidean_dist(p2,p6) + euclidean_dist(p3,p5)) / (2.0 * euclidean_dist(p1,p4))
    return ear

# ===============================
# STEP 2: Create folders
# ===============================
output_dir = "yolo_dataset"
alert_dir = os.path.join(output_dir, "Alert")
drowsy_dir = os.path.join(output_dir, "Drowsy")
os.makedirs(alert_dir, exist_ok=True)
os.makedirs(drowsy_dir, exist_ok=True)

# ===============================
# STEP 3: Process video
# ===============================
cap = cv2.VideoCapture("/content/Himanshu_New_Dataset.mp4")  # change to your video file
frame_count = 0
EAR_THRESHOLD = 0.25  # tweak if needed

with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        h, w, _ = frame.shape
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for landmarks in results.multi_face_landmarks:
                leftEAR = compute_EAR(landmarks.landmark, LEFT_EYE, w, h)
                rightEAR = compute_EAR(landmarks.landmark, RIGHT_EYE, w, h)
                EAR = (leftEAR + rightEAR) / 2.0

                # Classification (Alert or Drowsy)
                if EAR < EAR_THRESHOLD:
                    label = "Drowsy"
                    save_path = os.path.join(drowsy_dir, f"drowsy_{frame_count}.jpg")
                else:
                    label = "Alert"
                    save_path = os.path.join(alert_dir, f"alert_{frame_count}.jpg")

                # Save frame
                cv2.imwrite(save_path, frame)

        frame_count += 1

cap.release()
print("✅ Frames saved into dataset/Alert and dataset/Drowsy")


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


✅ Frames saved into dataset/Alert and dataset/Drowsy


Labels

In [ ]:
import os
import cv2
import mediapipe as mp
from glob import glob

# ===============================
# Config
# ===============================
dataset_dir = "/content/yolo_dataset"  # folder with Alert/ Drowsy
output_label_dir = "dataset_labels"
os.makedirs(output_label_dir, exist_ok=True)

# classes mapping
classes = {"Alert": 0, "Drowsy": 1}

# Eye landmark indices (for bounding box)
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# ===============================
# Functions
# ===============================
def get_face_bbox(landmarks, img_w, img_h, margin=0.2):
    xs = [lm.x * img_w for lm in landmarks]
    ys = [lm.y * img_h for lm in landmarks]
    x1, y1 = max(0, min(xs)), max(0, min(ys))
    x2, y2 = min(img_w - 1, max(xs)), min(img_h - 1, max(ys))

    # expand bbox by margin
    bw, bh = x2 - x1, y2 - y1
    x1 = max(0, x1 - bw * margin)
    y1 = max(0, y1 - bh * margin)
    x2 = min(img_w - 1, x2 + bw * margin)
    y2 = min(img_h - 1, y2 + bh * margin)

    return int(x1), int(y1), int(x2), int(y2)

def save_yolo_label(image_path, label_path, class_id, bbox):
    x1, y1, x2, y2 = bbox
    img = cv2.imread(image_path)
    h, w, _ = img.shape

    x_center = (x1 + x2)/2 / w
    y_center = (y1 + y2)/2 / h
    bw = (x2 - x1)/w
    bh = (y2 - y1)/h

    line = f"{class_id} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n"
    with open(label_path, "w") as f:
        f.write(line)

# ===============================
# Process all images
# ===============================
with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
    for cls_name in ["Alert", "Drowsy"]:
        cls_dir = os.path.join(dataset_dir, cls_name)
        images = glob(os.path.join(cls_dir, "*.jpg"))
        for img_path in images:
            img = cv2.imread(img_path)
            if img is None:
                continue
            h, w, _ = img.shape
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            results = face_mesh.process(rgb_img)
            if results.multi_face_landmarks:
                lms = results.multi_face_landmarks[0].landmark
                bbox = get_face_bbox(lms, w, h)
                # save label
                label_filename = os.path.basename(img_path).replace(".jpg", ".txt")
                label_path = os.path.join(output_label_dir, label_filename)
                save_yolo_label(img_path, label_path, classes[cls_name], bbox)

print("✅ YOLO annotation files created in:", output_label_dir)


✅ YOLO annotation files created in: dataset_labels


In [ ]:
import os
import glob
import shutil
import random

# ===============================
# Config
# ===============================
images_dir = "yolo_dataset"            # Your images: dataset/Alert & dataset/Drowsy
labels_dir = "dataset_labels"     # Your YOLO txt labels
output_dir = "yolo_final_output"       # Final folder for YOLOv8
random_seed = 42
val_ratio = 0.1
test_ratio = 0.1

classes = ["Alert", "Drowsy"]

os.makedirs(output_dir, exist_ok=True)

# ===============================
# Gather all images and labels
# ===============================
all_images = []
for cls in classes:
    cls_dir = os.path.join(images_dir, cls)
    imgs = glob.glob(os.path.join(cls_dir, "*.jpg"))
    for img_path in imgs:
        label_path = os.path.join(labels_dir, os.path.basename(img_path).replace(".jpg", ".txt"))
        if os.path.exists(label_path):
            all_images.append((img_path, label_path))

print(f"Total labeled images found: {len(all_images)}")

# ===============================
# Shuffle and split
# ===============================
random.seed(random_seed)
random.shuffle(all_images)

n_total = len(all_images)
n_val = int(n_total * val_ratio)
n_test = int(n_total * test_ratio)
n_train = n_total - n_val - n_test

train_set = all_images[:n_train]
val_set   = all_images[n_train:n_train+n_val]
test_set  = all_images[n_train+n_val:]

splits = {"train": train_set, "val": val_set, "test": test_set}

# ===============================
# Copy images and labels to YOLO structure
# ===============================
for split, dataset in splits.items():
    img_out = os.path.join(output_dir, "images", split)
    lbl_out = os.path.join(output_dir, "labels", split)
    os.makedirs(img_out, exist_ok=True)
    os.makedirs(lbl_out, exist_ok=True)

    for img_path, lbl_path in dataset:
        shutil.copy2(img_path, os.path.join(img_out, os.path.basename(img_path)))
        shutil.copy2(lbl_path, os.path.join(lbl_out, os.path.basename(lbl_path)))

# ===============================
# Create data.yaml
# ===============================
yaml_path = os.path.join(output_dir, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(f"train: {os.path.abspath(os.path.join(output_dir,'images','train'))}\n")
    f.write(f"val: {os.path.abspath(os.path.join(output_dir,'images','val'))}\n")
    f.write(f"test: {os.path.abspath(os.path.join(output_dir,'images','test'))}\n\n")
    f.write(f"nc: {len(classes)}\n")
    f.write(f"names: {classes}\n")

print(f"✅ YOLOv8 dataset ready! data.yaml created at: {yaml_path}")


Total labeled images found: 956
✅ YOLOv8 dataset ready! data.yaml created at: yolo_final_output/data.yaml


In [ ]:
import os

!yolo detect train model=yolov8n.pt data=yolo_final_output/data.yaml epochs=10 imgsz=640


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.193 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_final_output/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, 

In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())
!nvidia-smi


GPU available: False
/bin/bash: line 1: nvidia-smi: command not found
